In [1]:
import os
import sys
sys.path.append('../../../')

import numpy as np
import torch
import torch.nn as nn

import argparse
import json

from copy import deepcopy

import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

from src.simulator.utils import generate_model, get_client_dataset

In [2]:
def load_config(config_path, proj_name):
    PATH = os.path.join(config_path, f"config_{proj_name}.json")
    with open(PATH, 'r') as f:
        config = json.load(f)
    return config

def load_model(model_name, modelPATH, config, device):
    # device = torch.device('cpu')
    PATH = os.path.join(modelPATH, model_name)
    model_dict = torch.load(PATH, map_location=device)

    global_model = generate_model(config).to(device)
    
    if (config.agg_method != "Center") and (config.agg_method != "Local"):
        global_model.load_state_dict(model_dict['global_model'], strict=False)
        local_model_dict = model_dict['local_model']
        
    else:
        global_model.load_state_dict(model_dict['model'], strict=False)
        local_model_dict = None

    return global_model, local_model_dict

def load_model(model_name, modelPATH, config, device):
    # device = torch.device('cpu')
    PATH = os.path.join(modelPATH, model_name)
    model_dict = torch.load(PATH, map_location=device)

    global_model = generate_model(config).to(device)
    
    if (config.agg_method != "Center") and (config.agg_method != "Local"):
        global_model.load_state_dict(model_dict['global_model'], strict=False)
        local_model_dict = model_dict['local_model']
        
    else:
        global_model.load_state_dict(model_dict['model'], strict=False)
        local_model_dict = None

    return global_model, local_model_dict

In [3]:
basePATH = "/NFS/Users/moonsh/AdaptFL/ckpt/"

In [33]:
model_type_dict = {"Center": ["generous-sunset-1", -1], # No need to specify epoch
                   "Local": ["good-surf-1", -1], # No need to specify epoch
                   "FedAvg" : ["polished-wood-3", -1],
                   "FedProx": ["spring-sponge-10", -1],  
                   "MOON": ["fancy-disco-2", -1]
                   }

In [ ]:
agg_method = "FedAvg"
wandb_name = model_type_dict[agg_method][0]
client_idx = 8


ckptPATH = os.path.join(basePATH, agg_method, wandb_name)
config = load_config(ckptPATH, wandb_name)
config['batch_size'] = 32
config['num_workers'] = 2
config['nowandb'] = True
config = argparse.Namespace(**config)

model_name = f'{wandb_name}_best_model.pth'
glob_model = load_model(model_name, ckptPATH, config, torch.device('cpu'))
local_weight = glob_model[1][client_idx]

glob_model = glob_model[0]
# local_model = deepcopy(glob_model)
# local_model.load_state_dict(local_model, strict=False)

In [116]:
TestDataset = get_client_dataset(config, 10, 
                                 'Test', verbose=False, 
                                 get_info=False, PATH='/NFS/Users/moonsh/data/FLData/')

client_dataset = TestDataset[client_idx]
dataloader = DataLoader(client_dataset, batch_size=len(client_dataset))

full_batch_dataset = next(iter(dataloader))

In [ ]:
local_model = deepcopy(glob_model)
local_model.load_state_dict(local_weight, strict=False)

In [118]:
activation={'Local': {},
            'Global': {}}

In [119]:
def get_activation(model_name, layer_name):
    def hook(module, input, output):
        activation[model_name][layer_name] = output.detach()
    return hook


def register_hooks_for_model(model, model_name):
    hooks = []
    for name, layer in model.named_modules():
        if (isinstance(layer, nn.Conv3d) or isinstance(layer, nn.Linear)) and ('downsample' not in name):
            hook = layer.register_forward_hook(get_activation(model_name, name))
            hooks.append(hook)
    return hooks

def remove_hooks(hooks):
    for hook in hooks:
        hook.remove()

In [120]:
hooks_model_a = register_hooks_for_model(local_model, 'Local')
hooks_model_b = register_hooks_for_model(glob_model, 'Global')

In [121]:
loc_rep = local_model(full_batch_dataset[0]) # predict 값
glob_rep = glob_model(full_batch_dataset[0]) # predict 값

In [ ]:
num_layers = 0
for (layer_name, loc_output), glob_output in zip(activation['Local'].items(), activation['Global'].values()):
    print(layer_name)
    if "layer4.0.conv1" in layer_name:
        channel = loc_output.shape[1]
        break
    num_layers += 1

In [ ]:
num_show_ch = 16
interval_ch = channel // num_show_ch
channel_index_list = [i for i in range(0, channel, interval_ch)]
len(channel_index_list)

if len(channel_index_list) > num_show_ch:
    channel_index_list = channel_index_list[:num_show_ch]
len(channel_index_list)

In [137]:
activations = loc_output
activations2 = glob_output

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(16, 16))

for plot_idx, i in enumerate(channel_index_list):
    ax = axs[plot_idx // 4, plot_idx % 4]

    channel_activations = activations[:, i, :, :, :].flatten()
    channel_activations2 = activations2[:, i, :, :, :].flatten()

    # Kernel Density Estimation
    kde = gaussian_kde(channel_activations)
    activation_range = np.linspace(channel_activations.min(), channel_activations.max(), 100)
    kde_values = kde(activation_range)

    kde2 = gaussian_kde(channel_activations2)
    activation_range2 = np.linspace(channel_activations2.min(), channel_activations2.max(), 100)
    kde_values2 = kde2(activation_range2)

    # Plot KDE
    ax.plot(activation_range, kde_values, color='red', label='Local')
    ax.plot(activation_range2, kde_values2, color='blue', label='FedAvg')
    ax.set_title('KDE of Activations (Channel {})'.format(i))
    ax.set_xlabel('Activation Value')
    ax.set_ylabel('Probability Density')
    ax.legend()

plt.tight_layout()
plt.show()

remove_hooks(hooks_model_a)
remove_hooks(hooks_model_b)

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# 가상 fMRI 데이터 (T = 100, X = 64, Y = 64, Z = 64 크기의 4D 텐서)
T, X, Y, Z = 100, 64, 64, 64
fmri_data = np.random.randn(T, X, Y, Z)  # 예시 fMRI 데이터 생성

# 특정 Z 값에서 2D 평면 선택 (예: Z = 32)
z_plane = 32
voxel_2d_data = fmri_data[:, :, :, z_plane]  # X, Y 평면에서 Z = 32인 slice

# 특정 시간 프레임에서 X-Y 평면의 voxel 신경 활동 강도 추출 (T = 50)
time_frame = 50
voxel_intensity_2d = voxel_2d_data[time_frame]

# 2D joint distribution 계산을 위해 좌표와 강도를 결합 (flattening 하지 않음)
x_coords, y_coords = np.meshgrid(np.arange(X), np.arange(Y), indexing='ij')
xy_coords = np.vstack([x_coords.ravel(), y_coords.ravel()])
voxel_values = voxel_intensity_2d.ravel()

In [ ]:
import torch

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Gaussian 커널 함수 정의
def gaussian_kernel(distance, bandwidth):
    return torch.exp(-0.5 * (distance / bandwidth) ** 2) / (bandwidth * torch.sqrt(2 * torch.tensor(torch.pi, device=distance.device)))

# Kernel Density Estimation (KDE) 구현
def kde_torch(data, bandwidth):
    N = data.size(0)

    # 모든 점들 사이의 거리 계산 (전체 병렬 계산)
    # data의 각 점을 N x N x D로 확장하여 모든 점 사이의 거리를 계산
    distances = torch.cdist(data, data)  # (N, N) 형태의 거리 행렬 계산

    # Gaussian 커널을 적용한 KDE 계산
    kernels = gaussian_kernel(distances, bandwidth)
    density = kernels.mean(dim=1)  # 각 점에 대한 평균 밀도 계산
    
    return density

# 예시 데이터 생성 (100개의 3D 데이터 포인트)
data = torch.randn(16, 64, 128, 64).to(device)  # 데이터를 GPU로 전송

# Bandwidth 설정
bandwidth = 0.5

# KDE 계산 (GPU에서 병렬 처리)
density = kde_torch(data, bandwidth)

# 결과 출력 (다시 CPU로 전송)
print(density.cpu())




In [6]:
import torch

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = torch.randn(16, 64, 128, 64).to(device)  # 데이터를 GPU로 전송
distances = torch.cdist(data, data)

In [ ]:
distances.shape